### 套件安裝指令

1. pip install git+https://github.com/APCLab/jieba-tw.git
2. pip install scikit-learn
3. pip install torch transformers
4. pip install gensim
5. pip install scipy==1.10.1  

In [19]:
import pandas as pd
from zhon.hanzi import punctuation
import re

In [20]:
def full_to_half(s):
    # 將字符串 s 中的全形字符和標點符號轉換為半形。
    n = []
    for char in s:
        code = ord(char)
        if code == 0x3000:  # 全形空格直接轉換
            code = 32
        elif 0xFF01 <= code <= 0xFF5E:  # 全形字符（除空格）轉換成半形
            code -= 0xFEE0
        n.append(chr(code))
    return ''.join(n)

In [21]:
with open('./Data/stopwords_tc.txt', encoding='utf-8', mode='r') as f:
    stop_words = []
    for l in f:
        stop_words.append(l.strip())

In [22]:
gpt_food_post_df = pd.read_csv('./Data/ChatGPT/chatgpt_generated_articles.csv')
gpt_food_post_df

,title,content,prompt_system,prompt_user
0,[食記] 台北 辰壽司割烹~大閘蟹宴,[食記] 台北 辰壽司割烹~大閘蟹宴\n\n- 餐廳名稱：辰壽司割烹\n- 消費時間：202...,你是美食部落客,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...
1,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶\n\n◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢...,你是美食熱愛者,寫一篇「[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶」為標題的文章，風格：PTT發文風格，...
2,[食記] 新北林口｜林口三井的點點心,[食記] 新北林口｜林口三井的點點心\n\n大家好，這裡是小葵，今天來跟大家分享我最近造訪的...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 新北林口｜林口三井的點點心」為標題的文章，要包含以...
3,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯\n\n各位美食好手們，今日小編要介紹的是...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯」為標題的...
4,[食記] 桃園。麵屋虎千代,[食記] 桃園。麵屋虎千代\n\n餐廳名稱：麵屋虎千代\n地址：桃園市中壢區中園路123號\...,你是美食評論家,寫一篇「[食記] 桃園。麵屋虎千代」為標題的文章，風格：PTT發文風格，要有餐廳名稱、地址、...
...,...,...,...,...
1001,[食記] 屏東潮州-牛大福。屏東牛肉料理,[食記] 屏東潮州-牛大福。屏東牛肉料理\n\n大家好，這次要跟大家分享在屏東潮州區一家美味...,你是美食熱愛者,以PTT美食版發文風格，寫一篇「[食記] 屏東潮州-牛大福。屏東牛肉料理」為標題的文章，要包...
1002,[食記] 個人評比 肉次方VS夯下去,[食記] 個人評比 肉次方VS夯下去\n\n餐廳名稱：肉次方\n消費時間：2022/09/1...,你是美食熱愛者,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...
1003,[食記] 桃園龍潭 糧園茶藝客家小館,[食記] 桃園龍潭 糧園茶藝客家小館\n\n餐廳名稱：糧園茶藝客家小館\n消費時間：2022...,你是美食部落客,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...
1004,[食記] 台北信義安和 吉可頌丹麥專賣店,[食記] 台北信義安和 吉可頌丹麥專賣店\n\n消費時間：2021年6月\n地址：台北市信義...,你是美食熱愛者,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...


### Stopwords Removal  

In [23]:
def remove_urls_and_phones(text):
    """
    移除文字中的網址和電話號碼。
    """
    # 正則表達式匹配網址
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, '', text)

    # 正則表達式匹配電話號碼（適用於多種常見格式）
    phone_pattern = r'(\d{2,4}[-.\s]??\d{3,4}[-.\s]??\d{3,4}|\(\d{2,4}\)\s*\d{3,4}[-.\s]??\d{3,4}|\d{10,11})'
    text = re.sub(phone_pattern, '', text)

    return text

In [24]:
def remove_english(text):
    """
    移除文字中的所有英文字符。
    """
    # 正則表達式匹配所有英文字母和英文單詞
    pattern = r'[A-Za-z]+'
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [25]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # 表情符號
        "\U0001F300-\U0001F5FF"  # 符號和圖案
        "\U0001F680-\U0001F6FF"  # 交通和符號
        "\U0001F700-\U0001F77F"  # 藝術符號
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)

In [26]:
def load_dictionary(file_path):
    with open(file_path, encoding='utf-8', mode='r') as f:
        dictionary = [l.strip() for l in f]
    return dictionary

In [27]:
def remove_stopwords(text, stopwords):
    # 使用列表推導式過濾掉停用詞
    filtered_words = remove_urls_and_phones(text)
    filtered_words = remove_english(filtered_words)
    filtered_words = remove_emojis(filtered_words)
    filtered_words = "".join(c for c in filtered_words if c not in ('；','，','。','！','：','「','」','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"','\\', ' ', '‧','・','◢','◤','\n'))
    filtered_words = "".join(word for word in filtered_words if word not in stopwords)
    filtered_words = filtered_words.replace(u'\u3000', u' ')
    # 將過濾後的單詞列表重新組合成字符串
    return filtered_words

### Test

In [28]:
contents = gpt_food_post_df.loc[:, 'content']
contents

0       [食記] 台北 辰壽司割烹~大閘蟹宴\n\n- 餐廳名稱：辰壽司割烹\n- 消費時間：202...
1       [食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶\n\n◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢...
2       [食記] 新北林口｜林口三井的點點心\n\n大家好，這裡是小葵，今天來跟大家分享我最近造訪的...
3       [食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯\n\n各位美食好手們，今日小編要介紹的是...
4       [食記] 桃園。麵屋虎千代\n\n餐廳名稱：麵屋虎千代\n地址：桃園市中壢區中園路123號\...
                              ...                        
1001    [食記] 屏東潮州-牛大福。屏東牛肉料理\n\n大家好，這次要跟大家分享在屏東潮州區一家美味...
1002    [食記] 個人評比 肉次方VS夯下去\n\n餐廳名稱：肉次方\n消費時間：2022/09/1...
1003    [食記] 桃園龍潭 糧園茶藝客家小館\n\n餐廳名稱：糧園茶藝客家小館\n消費時間：2022...
1004    [食記] 台北信義安和 吉可頌丹麥專賣店\n\n消費時間：2021年6月\n地址：台北市信義...
1005    [食記] 高雄三民 早午餐-墨爾漢堡建工店\n\n餐廳名稱：墨爾漢堡建工店\n消費時間：20...
Name: content, Length: 1006, dtype: object

In [29]:
stop_words = load_dictionary('./Data/stopwords_tc.txt')
remove_stopword_contents = []
for c in contents:
    remove_stopword_contents.append(remove_stopwords(c, stop_words))
gpt_food_post_df['remove_stopword_content'] = remove_stopword_contents
gpt_food_post_df

,title,content,prompt_system,prompt_user,remove_stopword_content
0,[食記] 台北 辰壽司割烹~大閘蟹宴,[食記] 台北 辰壽司割烹~大閘蟹宴\n\n- 餐廳名稱：辰壽司割烹\n- 消費時間：202...,你是美食部落客,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...,食記台北辰壽司割烹閘蟹宴餐廳名稱辰壽司割烹消費時間年月址台北市信義區松壽路號電話營業時間天台...
1,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶\n\n◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢...,你是美食熱愛者,寫一篇「[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶」為標題的文章，風格：PTT發文風格，...,食記新北金鋤壽喜燒烤鍋物新莊店午茶餐廳名稱金鋤壽喜燒烤鍋物新莊店址新北市新莊區中港路號電話營...
2,[食記] 新北林口｜林口三井的點點心,[食記] 新北林口｜林口三井的點點心\n\n大家好，這裡是小葵，今天來跟大家分享我最近造訪的...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 新北林口｜林口三井的點點心」為標題的文章，要包含以...,食記新北林口林口井點點心家裡葵天家分享造訪家美食店新北市林口區林口井購物中心家店做點點心址新...
3,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯\n\n各位美食好手們，今日小編要介紹的是...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯」為標題的...,食記台北松山夯魯肉飯原分埔魯肉飯位美食手日編介紹位台北松山區夯魯肉飯原分埔魯肉飯家餐廳魯肉飯...
4,[食記] 桃園。麵屋虎千代,[食記] 桃園。麵屋虎千代\n\n餐廳名稱：麵屋虎千代\n地址：桃園市中壢區中園路123號\...,你是美食評論家,寫一篇「[食記] 桃園。麵屋虎千代」為標題的文章，風格：PTT發文風格，要有餐廳名稱、地址、...,食記桃園麵屋虎代餐廳名稱麵屋虎代址桃園市中壢區中園路號電話營業時間週週日價位平消費約元消費日...
...,...,...,...,...,...
1001,[食記] 屏東潮州-牛大福。屏東牛肉料理,[食記] 屏東潮州-牛大福。屏東牛肉料理\n\n大家好，這次要跟大家分享在屏東潮州區一家美味...,你是美食熱愛者,以PTT美食版發文風格，寫一篇「[食記] 屏東潮州-牛大福。屏東牛肉料理」為標題的文章，要包...,食記屏東潮州牛福屏東牛肉料理家次家分享屏東潮州區家美味牛肉料理餐廳牛福裡品嚐道美味牛肉料理回...
1002,[食記] 個人評比 肉次方VS夯下去,[食記] 個人評比 肉次方VS夯下去\n\n餐廳名稱：肉次方\n消費時間：2022/09/1...,你是美食熱愛者,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...,食記評肉次夯餐廳名稱肉次消費時間址台北市安區復興南路段號電話營業時間週週日進入肉次空間刻溫暖...
1003,[食記] 桃園龍潭 糧園茶藝客家小館,[食記] 桃園龍潭 糧園茶藝客家小館\n\n餐廳名稱：糧園茶藝客家小館\n消費時間：2022...,你是美食部落客,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...,食記桃園龍潭糧園茶藝客家館餐廳名稱糧園茶藝客家館消費時間址桃園市龍潭區路號電話營業時間週週日...
1004,[食記] 台北信義安和 吉可頌丹麥專賣店,[食記] 台北信義安和 吉可頌丹麥專賣店\n\n消費時間：2021年6月\n地址：台北市信義...,你是美食熱愛者,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...,食記台北信義安吉頌丹麥專賣店消費時間年月址台北市信義區松仁路號電話營業時間週週日天台北信義區...


### Vectorlize

In [30]:
remove_stopword_contents = gpt_food_post_df['remove_stopword_content']
remove_stopword_contents_list = remove_stopword_contents.to_list()
remove_stopword_contents_list

['食記台北辰壽司割烹閘蟹宴餐廳名稱辰壽司割烹消費時間年月址台北市信義區松壽路號電話營業時間天台北辰壽司割烹專門提供美味壽司割烹料理餐廳裝潢雅營造舒適餐環境感受高雅氛圍店員服務非親周點推薦閘蟹宴首先桌清爽閘蟹沙拉鮮甜蟹肉搭配蔬菜開胃美味接閘蟹壽司口充濃郁蟹味令賞心悅目精心烹調閘蟹割烹料理搭配新鮮蔬菜香料口感豐富樣回味窮整體說辰壽司割烹閘蟹宴物超值食物新鮮美味份量十足吃足舒服價格偏高考量食材品質餐廳服務非值美食體驗果喜愛日料理強烈推薦辰壽司割烹品嚐閘蟹宴',
 '食記新北金鋤壽喜燒烤鍋物新莊店午茶餐廳名稱金鋤壽喜燒烤鍋物新莊店址新北市新莊區中港路號電話營業時間平日週末價位中價位消費付消費日期天午朋友決新莊找間餐廳享午茶時偶發現金鋤壽喜燒烤鍋物家餐廳位新莊區中港路簡樸外觀眼感受舒適餐氛圍走進店首先烤肉香吸引住胃口開店裝潢簡潔舒適服務員十分親熱心選擇享受午茶時點道燒烤拼盤鍋物清爽沙拉冰涼飲料燒烤拼盤桌時烤肉金黃鮮嫩肉質鮮美汁搭配特製醬料美味鍋物湯底清香濃郁滋味十足沙拉新鮮爽口口清爽幸福整體說美味料理度愉午茶時金鋤壽喜燒烤鍋物享受美食感受溫暖服務放鬆環境次機餐享受美食驚喜強力推薦喜歡日式料理朋友推薦家嚐嚐特製醬料愛釋手',
 '食記新北林口林口井點點心家裡葵天家分享造訪家美食店新北市林口區林口井購物中心家店做點點心址新北市林口區文化路段號☎️電話營業時間週週日價位平點點心簡約舒適裝潢風格吸引少食客店提供款典台式吃甜點選擇欲罷推薦家統鮮肉包外皮彈軟綿包裹鮮美香氣餡料口咬感覺幸福外花生湯圓驚艷道點心湯圓彈口濃厚花生香氣令回味窮想午輕食找細細品味台式吃林口井點點心容錯處次餐心分享希家店親品嚐感受裡美味果問題想法歡迎面留言家交流討詳細資訊參考點點心官網站臉書粉絲專頁',
 '食記台北松山夯魯肉飯原分埔魯肉飯位美食手日編介紹位台北松山區夯魯肉飯原分埔魯肉飯家餐廳魯肉飯愛訪聖探訪餐廳名稱夯魯肉飯原分埔魯肉飯址台北市松山區民權東路段號電話營業時間週週日價位碗魯肉飯濟實惠美味口家餐廳魯肉飯統分埔口味基礎口感鮮嫩汁甜中絲鹹香吃回味窮搭配顆滷蛋香噴噴醃漬青菜簡配菜搭配口充幸福感外店服務態度十分親吃心意足位美食愛果想趟美味濟實惠魯肉飯旅萬錯家夯魯肉飯原分埔魯肉飯記訪時記點碗欲罷編品味份風味特台灣味道é',
 '食記桃園麵屋虎代餐廳名稱麵屋虎代址桃園市中壢區中園路號電話營業時間週週日價位平消費約元

In [31]:
import jieba
# 使用 Jieba 進行切詞
def jieba_cut(text):
    return ' '.join(jieba.cut(text))

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_vectorlize(posts):

    # 將每篇文章進行切詞
    cut_posts = [jieba_cut(post) for post in posts]

    # 初始化 TfidfVectorizer，這裡可以調整參數
    vectorizer = TfidfVectorizer()

    # 將切詞後的文章內容轉換為 TF-IDF 矩陣
    tfidf_matrix = vectorizer.fit_transform(cut_posts)

    # 獲取特徵名稱（詞彙）
    feature_names = vectorizer.get_feature_names_out()

    # 將 TF-IDF 矩陣轉換為 DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=range(len(posts)), columns=feature_names)

    return tfidf_df

In [33]:
tfidf_vectorlize(remove_stopword_contents_list).to_csv('./Data/ChatGPT/gpt_tfidf_vector_back.csv')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/rh/7g0n3djn3wj8dlnjv7plmzcr0000gn/T/jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built succesfully.


In [34]:
import gensim
import numpy as np

def word2vec_vectorlize(posts):
    model = gensim.models.KeyedVectors.load_word2vec_format('./IgnoreFile/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
                                                        unicode_errors='ignore', 
                                                        binary=True)
    
    # 將每篇文章進行切詞
    result = []
    cut_posts = [jieba_cut(post) for post in posts]
    for cut_post in cut_posts:
        words = cut_post.split()
        vector_size = model.vector_size
        sentence_vector = np.zeros(vector_size)
    
        count = 0
        for word in words:
            if word in model:
                sentence_vector += model[word]
                count += 1
        if count > 0:
            sentence_vector /= count
        result.append(sentence_vector)
    
    return result

In [35]:
word2vec_vecotrs = word2vec_vectorlize(remove_stopword_contents_list)
word2vec_df = pd.DataFrame(word2vec_vecotrs, columns=[f'v{i}' for i in range(len(word2vec_vecotrs[0]))])
word2vec_df.to_csv('./Data/ChatGPT/gpt_word2vec_vector_back.csv')

In [36]:
from transformers import BertTokenizer, BertModel
import torch

def bert_vectorlize(posts):
    # 初始化 BERT 分詞器和模型
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    model = BertModel.from_pretrained('bert-base-chinese')

    # 將每篇文章進行切詞
    result = []
    cut_posts = [jieba_cut(post) for post in posts]

    for cut_post in cut_posts:
        inputs = tokenizer(cut_post, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        sentence_embedding = outputs.last_hidden_state.mean(dim=1)
        result.append(sentence_embedding)
    return result

/Users/jim/anaconda3/envs/forPttCrawler/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
bert_vectors = bert_vectorlize(remove_stopword_contents_list)
bert_df = pd.DataFrame(torch.cat(bert_vectors).numpy(), columns=[f'v{i}' for i in range(len(bert_vectors[0][0]))])
bert_df.to_csv('./Data/ChatGPT/gpt_bert_vector_back.csv')

/Users/jim/anaconda3/envs/forPttCrawler/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
